In [1]:
from pathlib import Path

import cv2
import matplotlib as plt
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

In [2]:
# def resizeAndPad(img, size, padColor=0):

#     h, w = img.shape[:2]
#     sh, sw = size

#     # interpolation method
#     if h > sh or w > sw: # shrinking image
#         interp = cv2.INTER_AREA
#     else: # stretching image
#         interp = cv2.INTER_CUBIC

#     # aspect ratio of image
#     aspect = w/h 

#     # compute scaling and pad sizing
#     if aspect > 1: # horizontal image
#         new_w = sw
#         new_h = np.round(new_w/aspect).astype(int)
#         pad_vert = (sh-new_h)/2
#         pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
#         pad_left, pad_right = 0, 0
#     elif aspect < 1: # vertical image
#         new_h = sh
#         new_w = np.round(new_h*aspect).astype(int)
#         pad_horz = (sw-new_w)/2
#         pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
#         pad_top, pad_bot = 0, 0
#     else: # square image
#         new_h, new_w = sh, sw
#         pad_left, pad_right, pad_top, pad_bot = 0, 0, 0, 0

#     # set pad color
#     if len(img.shape) is 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided
#         padColor = [padColor]*3

#     # scale and pad
#     scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
#     scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

#     return scaled_img

In [3]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [4]:
def getROI(filepath, size, asix, slice):

    mri_img = nib.load(filepath)
    n1_header = mri_img.header

    pixdim = mri_img.header['pixdim'][1:4]
    dim = mri_img.header['dim'][1:4]

    img_data = mri_img.get_fdata()

  

    new_mri_img = zoom(img_data, pixdim)
    r, a, s = new_mri_img.shape
    if 'T2' in filepath:
        if asix == 'r':
            img_normalized = cv2.normalize(new_mri_img[:, :, int(s / 2) - 1 + slice], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
        if asix == 'a':
            img_normalized = cv2.normalize(new_mri_img[int(r / 2) - 1 + slice, :, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_180)
            img_normalized = cv2.flip(img_normalized, 1)
        if asix == 's':
            img_normalized = cv2.normalize(new_mri_img[:, int(a / 2) - 1 + slice, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.flip(img_normalized, 1)
    else:
        if asix == 'r':
            img_normalized = cv2.normalize(new_mri_img[int(r / 2) - 1 + slice, :, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
            img_normalized = cv2.flip(img_normalized, 1)
        if asix == 'a':
            img_normalized = cv2.normalize(new_mri_img[:, int(a / 2) - 1 + slice, :], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
            img_normalized = cv2.flip(img_normalized, 1)
        if asix == 's':
            img_normalized = cv2.normalize(new_mri_img[:, :, int(s / 2) - 1 + slice], None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            img_normalized = cv2.rotate(img_normalized, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
    img = cv2.merge([img_normalized, img_normalized, img_normalized])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if 'DTI' not in filepath:
        img_bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] #+ cv2.THRESH_OTSU
    else:
        img_bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] #+ cv2.THRESH_OTSU

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))
    erosion = cv2.dilate(img_bw,kernel,iterations = 1)

    cnts = cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    c = max(cnts, key = cv2.contourArea)
   
    x, y, w, h = cv2.boundingRect(c)
    #cv2.rectangle(img, (x, y), (x + w, y + h), color=(255, 0, 0), thickness=1)
    x_new = x - 3 if x - 3 > 0 else 0
    y_new = y - 5 if y - 5 > 0 else 0
    w_new = w + 3 if w + 3 < img.shape[0] else img.shape[0]
    h_new = h + 5 if h + 5 < img.shape[1] else img.shape[1]
    img_crop = img[y_new:y+h_new, x_new:x+w_new]
    if h > w:
        img_resize = image_resize(img_crop, height = size[1])
    else:
        img_resize = image_resize(img_crop, width = size[0])
    top_size, bottom_size, left_size, right_size = (50, 50, 50, 50)
    replicate = cv2.copyMakeBorder(img_resize, top_size, bottom_size, left_size, right_size, cv2.BORDER_REPLICATE)
    
    
    x = replicate.shape[1] / 2 - size[1]/2
    y = replicate.shape[0] / 2 - size[0]/2
    w = size[0]
    h = size[1]
    img_resize_crop = replicate[int(y):int(y+h), int(x):int(x+w)]
    #scaled_img = resizeAndPad(img_crop, size)
    #cv2.normalize(img_resize_crop, img_resize_crop, 0, 1, cv2.NORM_MINMAX)
    #img_resize_crop *= 255
    maxindex  = img_resize_crop.max()
    minindex = img_resize_crop.min()
    print('resize image max'+ str(maxindex) + 'min'+str(minindex))
    return img_resize_crop

In [5]:

# scaled_img = getROI('./ixi_augmentation/T1/RandomMotion/RM-IXI002-Guys-0828-T1.nii.gz', (64,64), 's', 0)
# imgplot = plt.imshow(scaled_img)
# plt.show()

In [6]:
import ruamel.yaml
yaml = ruamel.yaml.YAML()
yaml.representer.ignore_aliases = lambda *data: True

DefaultConfigPath = "Config.yaml"

with open(DefaultConfigPath, encoding="utf-8") as inyaml:
    yaml_data = yaml.load(inyaml)

In [7]:
from pathlib import Path
import os
for key in yaml_data["modalities"]:
    for key1 in yaml_data["transform"]:
        folder = Path(yaml_data["output_slice_dir_name"], key, key1)
        if not os.path.exists(folder): 
            os.makedirs(folder)
        dataset_dir = Path(yaml_data["output_augmentation_dir_name"])
        images_dir = dataset_dir / key / key1

        for root,dirs,names in os.walk(images_dir):
            for filename in names:
                if filename.endswith('.nii.gz') and '._' not in filename:
                    for slice in yaml_data["slice"]:
                        slice_name = filename.rstrip('.nii.gz') + yaml_data["asix"] +'_' +str(slice) + '.png'
                        folder = Path(yaml_data["output_slice_dir_name"], key, key1, filename.rstrip('.nii.gz'))
                        if not os.path.exists(folder): 
                            os.makedirs(folder)
                        scaled = getROI(os.path.join(root,filename), (64,64), yaml_data["asix"], slice)
                        cv2.imwrite(os.path.join(folder,slice_name), scaled)
                    #print(os.path.join(root,filename)) 
        #image_paths = sorted(images_dir.glob('*.nii.gz'))
        #scaled_img = getROI(image_paths, (64,64), 's', 0)


resize image max173min0
resize image max202min2
resize image max224min0
resize image max247min2
resize image max203min0
resize image max243min2
resize image max199min5
resize image max227min1
resize image max210min1
resize image max230min3
resize image max207min4
resize image max216min3
resize image max202min1
resize image max223min3
resize image max217min2
resize image max206min2
resize image max237min3
resize image max214min3
resize image max204min3
resize image max200min2
resize image max213min4
resize image max210min2
resize image max201min3
resize image max214min3
resize image max211min3
resize image max204min1
resize image max226min2
resize image max211min1
resize image max230min4
resize image max247min3
resize image max248min0
resize image max189min2
resize image max198min5
resize image max203min2
resize image max249min2
resize image max246min0
resize image max225min3
resize image max205min5
resize image max221min2
resize image max212min4
resize image max213min6
resize image max